## Notebook 3: Further Data Transformation  

This notebook third stage of the data engineering challenge
It performs three main steps:  
1. **Subsampling**: Select service requests near the centroid of Bellville South.  
2. **Augmentation**: Enrich requests with wind data from the Bellville South AQM site.  
3. **Anonymisation**: Apply spatial and temporal precision limits while removing sensitive identifiers.  

In [ ]:
from __future__ import annotations

# --- Stdlib ---
import contextlib
import functools
import gzip
import io
import json
import logging
import os
import re
import shutil
import time
import types
from pathlib import Path
from typing import Iterable, Optional
from pyproj import Transformer

# --- Main ---
import __main__ as nb

# --- Data Science ---
import numpy as np
import pandas as pd
import geopandas as gpd

# --- HTTP ---
import requests

# --- Geo ---
from shapely.geometry import shape
from shapely.geometry.base import BaseGeometry

# --- Testing ---
import ipytest
import pytest

# --- Optional resources ---
try:
    import psutil as _psutil
except Exception:
    _psutil = None

try:
    import tracemalloc as _tracemalloc
except Exception:
    _tracemalloc = None

# --- Config ---
ipytest.autoconfig()

# --- Logging setup ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# --- Utilities ---
def ensure_dir(path: str | os.PathLike) -> None:
    os.makedirs(str(path), exist_ok=True)

def parent_dir_of(path: str | os.PathLike) -> str:
    return os.path.dirname(str(path)) or "."

def fmt_bytes(n: int | float) -> str:
    try:
        n = float(n)
    except Exception:
        return str(n)
    for unit in ("B", "KB", "MB", "GB", "TB", "PB"):
        if abs(n) < 1024 or unit == "PB":
            return f"{n:.1f}{unit}"
        n /= 1024

def gunzip_file(src: str, dst: str) -> None:
    ensure_dir(parent_dir_of(dst))
    with gzip.open(src, "rb") as fin, open(dst, "wb") as fout:
        shutil.copyfileobj(fin, fout)

def client_error_msg(e) -> str:
    code = getattr(e, "response", {}).get("Error", {}).get("Code")
    msg  = getattr(e, "response", {}).get("Error", {}).get("Message")
    return f"[{code}] {msg}" if code or msg else str(e)

@contextlib.contextmanager
def timed(label: str, enabled: bool = True):
    t0 = time.perf_counter()
    try:
        yield
    finally:
        if enabled:
            logger.info("%s took %.3f s", label, time.perf_counter() - t0)

def resource_snapshot(note: str = "") -> None:
    if not logger.isEnabledFor(logging.DEBUG):
        return
    parts: list[str] = []
    if _psutil:
        try:
            p = _psutil.Process(os.getpid())
            parts.append(f"rss={fmt_bytes(p.memory_info().rss)}")
            parts.append(f"cpu%~{p.cpu_percent(interval=0.0):.1f}")
        except Exception:
            pass
    if _tracemalloc and _tracemalloc.is_tracing():
        try:
            cur, peak = _tracemalloc.get_traced_memory()
            parts.append(f"py_mem={fmt_bytes(cur)}/{fmt_bytes(peak)}(peak)")
        except Exception:
            pass
    if parts:
        logger.debug("RES%s %s", f"[{note}]" if note else "", " ".join(parts))

# --- Project defaults ---
DEST_ROOT = "../data"

def resolve_path(rel: str | None, filename: str) -> Path:
    return Path(rel) / filename if rel else Path(DEST_ROOT) / filename

def load_project_files(
    file_map: dict[str, str],
    project_root_name: str = "ds_code_challenge",
    inject_globals: bool = True,
):
    """
    Load files from ROOT/data based on a {file_name: variable_name} map.

    Supports:
      - .csv / .csv.gz  -> pandas DataFrame
      - .geojson / .geojson.gz -> GeoDataFrame
      - .ods -> Excel (odf engine)
    """
    with timed("load_project_files (resolve ROOT/DATA_DIR)"):
        ROOT = Path(__file__).resolve().parents[0] if "__file__" in globals() else Path().resolve()
        while ROOT.name != project_root_name and ROOT.parent != ROOT:
            ROOT = ROOT.parent
        DATA_DIR = ROOT / "data"

    results: dict[str, pd.DataFrame | gpd.GeoDataFrame] = {}
    resource_snapshot("load_project_files:start")

    for file_name, var_name in file_map.items():
        file_path = DATA_DIR / file_name
        logger.info("Processing %s...", file_path)

        with timed(f"read:{file_path.name}"):
            suffix = "".join(file_path.suffixes).lower()

            if suffix.endswith((".csv", ".csv.gz")):
                df = pd.read_csv(file_path)
                results[var_name] = df
                logger.info("→ %s loaded as DataFrame shape=%s", var_name, getattr(df, "shape", None))

            elif suffix.endswith((".geojson", ".geojson.gz")):
                gdf = gpd.read_file(file_path)
                results[var_name] = gdf
                logger.info("→ %s loaded as GeoDataFrame len=%d", var_name, len(gdf))

            elif suffix.endswith(".ods"):
                df = pd.read_excel(file_path, engine="odf")
                results[var_name] = df
                logger.info("→ %s loaded from ODS shape=%s", var_name, getattr(df, "shape", None))

            else:
                logger.warning("Skipping unsupported file type: %s", file_path)
                continue

            if inject_globals:
                globals()[var_name] = results[var_name]

            resource_snapshot(f"after_load:{file_path.name}")

    logger.info("All files loaded successfully.")
    resource_snapshot("load_project_files:end")
    return results





In [ ]:
def save_df_to_csv(df: pd.DataFrame, file_name: str, index: bool = False):
    """
    Saves a DataFrame to a CSV file in the project's data directory.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        file_name (str): The name of the output file (e.g., 'anonymized_data.csv').
        index (bool, optional): Whether to write row names (index). Defaults to False.
    """
    # Define the root directory of your project
    ROOT = Path.cwd().parent
    
    # Define the data directory and ensure it exists
    DATA_DIR = ROOT / "output"
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    
    # Ensure the filename ends with .csv
    if not file_name.endswith('.csv'):
        file_name += '.csv'
        
    file_path = DATA_DIR / file_name
    
    # Save the DataFrame to CSV with configurable index
    df.to_csv(file_path, index=index)
    
    print(f"DataFrame successfully saved to: {file_path}")

In [ ]:


def download_file(url: str, filename: str = "Wind_direction_and_speed_2020.ods", rel: str | None = None):
    """Download a file into project data dir."""
    save_path = resolve_path(rel, filename)
    ensure_dir(save_path.parent)

    with timed(f"download:{filename}"):
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            bytes_written = 0
            with open(save_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        bytes_written += len(chunk)

        logger.info("File downloaded to %s (%s)", save_path.resolve(), fmt_bytes(bytes_written))
        resource_snapshot(f"after_download:{filename}")


# Downloads wind and speed direction ods

In [ ]:
url = "https://www.capetown.gov.za/_layouts/OpenDataPortalHandler/DownloadHandler.ashx?DocumentName=Wind_direction_and_speed_2020.ods&DatasetDocument=https%3A%2F%2Fcityapps.capetown.gov.za%2Fsites%2Fopendatacatalog%2FDocuments%2FWind%2FWind_direction_and_speed_2020.ods"
download_file(url)

In [ ]:
def fetch_centroid_latlon(
    suburb_name: str,
    *,
    featureserver_url: str,
    epsg_utm: int | None = 32734,
    req_timeout: int = 60,
) -> tuple[float, float]:
    """Return (lat, lon) centroid for official suburb polygon."""
    params = {
        "where": f"OFC_SBRB_NAME='{suburb_name}'",
        "outFields": "*",
        "returnGeometry": "true",
        "outSR": "4326",
        "f": "geojson",
    }

    with timed(f"fetch_centroid_latlon[{suburb_name}]"):
        r = requests.get(featureserver_url, params=params, timeout=req_timeout)
        r.raise_for_status()
        gj = r.json()
        resource_snapshot("after_download")

        feats = gj.get("features", [])
        if not feats:
            raise RuntimeError(f"No polygon returned for suburb='{suburb_name}'")

        gdf = gpd.GeoDataFrame.from_features(feats, crs="EPSG:4326")
        if len(gdf) > 1:
            gdf["ones"] = 1
            gdf = gdf.dissolve(by="ones", as_index=False, aggfunc="first").drop(columns="ones")

        # pick a projected CRS for accurate centroid if needed
        def _auto_utm_epsg(gdf_ll: gpd.GeoDataFrame) -> int:
            c_ll = gdf_ll.unary_union.centroid
            lon0, lat0 = float(c_ll.x), float(c_ll.y)
            zone = int((lon0 + 180) // 6 + 1)
            return (32600 + zone) if lat0 >= 0 else (32700 + zone)

        epsg_proj = (epsg_utm if epsg_utm not in (None, 4326) else _auto_utm_epsg(gdf))
        if epsg_utm in (None, 4326):
            logger.info("Using projected CRS EPSG:%d for centroid", epsg_proj)

        # project polygon → centroid in meters
        gdf_proj = gdf.to_crs(epsg=epsg_proj)
        c_proj = gdf_proj.geometry.centroid.iloc[0]  # projected centroid (meters)

        # transform that single point back to WGS84 as true scalars
        transformer = Transformer.from_crs(epsg_proj, 4326, always_xy=True)
        cent_lon, cent_lat = transformer.transform(float(c_proj.x), float(c_proj.y))

        logger.info("Centroid = (%.6f, %.6f)", cent_lat, cent_lon)
        resource_snapshot("after_centroid")

    return float(cent_lat), float(cent_lon)



In [ ]:
file_map = {
    "Wind_direction_and_speed_2020.ods": "df_wind",
    "sr_hex.csv": "df_sr_hex",
}

results = load_project_files(file_map)

# Access DataFrames from results (and from globals if inject_globals=True)
df_wind   = results["df_wind"]
df_sr_hex = results["df_sr_hex"]


In [ ]:
def haversine_km(lat1, lon1, lat2, lon2):
    """Vectorized great-circle distance in km."""
    R = 6371.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi    = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def filter_within_radius_km(

    df_latlon: pd.DataFrame,
    cent_lat: float,
    cent_lon: float,
    *,
    radius_km: float = 1.852,
    lat_col: str = "latitude",
    lon_col: str = "longitude",
    keep_distance_col: bool = True,
) -> pd.DataFrame:
    """
    Filters out service requests beyond a certain radius from the centroid
    """
    with timed(f"filter_within_radius_km[{radius_km:.3f} km]"):
        mask_valid = df_latlon[lat_col].notna() & df_latlon[lon_col].notna()
        n_invalid = (~mask_valid).sum()
        if n_invalid:
            logger.info("Skipped %d rows with missing coordinates", int(n_invalid))

        df = df_latlon.loc[mask_valid].copy()
        df["distance_km"] = haversine_km(
            df[lat_col].astype(float).values,
            df[lon_col].astype(float).values,
            cent_lat, cent_lon
        )
        out = df.loc[df["distance_km"] <= radius_km].copy()
        if not keep_distance_col:
            out.drop(columns=["distance_km"], inplace=True)

        logger.info("Kept %d/%d valid rows within %.3f km",
                    len(out), len(df), radius_km)
        resource_snapshot("after_radius_filter")
    return out



# Created subsample of sr_hex.csv and get centroid
This was done by using the suburb’s official name to pull its polygon from the City of Cape Town’s Map Viewer. From there, I used geometry.centroid to extract the centroid latitude and longitude. I then defined a circle around that point based on 1 minute of latitude (≈ 1.852 km). Latitude was chosen since it’s a consistent angular measure globally, whereas longitude varies by latitude. Finally, I calculated the vectorized haversine distance between the centroid and each service request in the CSV, and filtered down to those falling within 1.852 km.


In [ ]:
FEATURESERVER_URL = "https://citymaps.capetown.gov.za/agsext/rest/services/Theme_Based/ODP_SPLIT_5/FeatureServer/3/query"
cent_lat, cent_lon = fetch_centroid_latlon("BELLVILLE SOUTH", featureserver_url=FEATURESERVER_URL)
df_1min_bellville = filter_within_radius_km(df_sr_hex, cent_lat, cent_lon)
save_df_to_csv(df_1min_bellville, "1min_bellville.csv")

In [ ]:


def _infer_site_wind_cols(df: pd.DataFrame, site_name: str) -> tuple[str, str]:
    cols = df.columns.astype(str)
    site_re = re.escape(site_name)
    dir_re = re.compile(rf"(?i){site_re}.*_Deg$")
    spd_re = re.compile(rf"(?i){site_re}.*_(m/s|m_s|ms)$")

    dir_candidates = [c for c in cols if dir_re.search(c)]
    spd_candidates = [c for c in cols if spd_re.search(c)]

    if not dir_candidates:
        dir_candidates = [c for c in cols if re.search(rf"(?i){site_re}.*deg", c)]
    if not spd_candidates:
        spd_candidates = [c for c in cols if re.search(rf"(?i){site_re}.*(m/s|m_s|ms|speed)", c)]

    if not dir_candidates or not spd_candidates:
        raise ValueError(
            f"Could not infer wind columns for site '{site_name}'. "
            f"Found direction candidates: {dir_candidates or '[]'}, "
            f"speed candidates: {spd_candidates or '[]'}"
        )

    dir_col = sorted(dir_candidates, key=len)[0]
    spd_col = sorted(spd_candidates, key=len)[0]
    logging.info("Inferred columns for '%s' -> dir: %s | spd: %s", site_name, dir_col, spd_col)
    return dir_col, spd_col

def process_wind_data(file_path: str, site_name: str) -> pd.DataFrame:
    """
    Process wind data for a given site.
    
    Parameters
    ----------
    file_path : str
        Path to the Excel/ODS file.
    site_name : str
        Name (or part of it) of the site to filter columns on (case-insensitive).
    
    Returns
    -------
    pd.DataFrame
        Processed DataFrame with DateTime index and site-specific columns.
    """
    t0 = time.perf_counter()
    logging.info("Processing: %s", file_path)

    # 1) Read file with 2-row header
    df = pd.read_excel(file_path, engine="odf", header=[1, 2], skiprows=[0, 3])

    # 2) Flatten header
    df.columns = pd.Index([
        (str(a).strip() if (pd.isna(b) or "Unnamed" in str(b))
         else f"{str(a).strip()}_{str(b).strip()}")
        for a, b in df.columns
    ]).str.replace(" ", "_", regex=False)

    # 3) Keep first DateTime column only
    dt_col = next(c for c in df.columns if "date" in c.lower() and "time" in c.lower())
    df["DateTime"] = pd.to_datetime(df[dt_col], errors="coerce", dayfirst=True)

    # 4) Keep DateTime + site-specific cols only, force numeric
    logger.debug(df.head())
    keep_mask = df.columns.str.contains(site_name, case=False) | (df.columns == "DateTime")
    df = df.loc[:, keep_mask]
    site_cols = df.columns[df.columns.str.contains(site_name, case=False)]
    df[site_cols] = df[site_cols].apply(pd.to_numeric, errors="coerce")

    # 5) Drop rows where DateTime is NaT
    n_before = len(df)
    df = df.dropna(subset=["DateTime"])
    n_dropped = n_before - len(df)
    if n_dropped:
        logging.info("Dropped %d rows with NaT DateTime", n_dropped)

    # 6) Deduplicate and set DateTime index
    df = df.drop_duplicates(subset=["DateTime"]).set_index("DateTime").sort_index()

    logging.info("Done. Shape: %s. Elapsed: %.3fs", df.shape, time.perf_counter() - t0)
    return df

# --- New: rename to generic labels ---
def standardize_wind_column_names(
    df: pd.DataFrame,
    site_name: str,
    dir_label: str = "wind direction degree",
    spd_label: str = "wind speed m/s",
) -> pd.DataFrame:
    """
    Locate a site's wind direction/speed columns and rename them to generic labels.
    Returns a copy with columns renamed.
    """
    df2 = df.copy()
    dir_col, spd_col = _infer_site_wind_cols(df2, site_name)

    # Build a safe, minimal mapping
    rename_map = {dir_col: dir_label, spd_col: spd_label}
    df2 = df2.rename(columns=rename_map)

    logging.info("Renamed '%s' -> '%s', '%s' -> '%s'", dir_col, dir_label, spd_col, spd_label)
    return df2

# --- Updated: fill uses standardized labels by default ---
def fill_wind_missing(df: pd.DataFrame,
                      dir_col: str = "wind direction degree",
                      spd_col: str = "wind speed m/s") -> pd.DataFrame:
    """Fills in missing wind speed and direction data with averages"""
    with timed("fill_wind_missing"):
        if not np.issubdtype(df.index.dtype, np.datetime64):
            raise ValueError("Index must be datetime for interpolation.")

        before = df[[dir_col, spd_col]].isna().sum()
        logger.info("NaNs before | %s=%d, %s=%d",
                    dir_col, int(before[dir_col]),
                    spd_col, int(before[spd_col]))

        # Speed interpolation
        df[spd_col] = pd.to_numeric(df[spd_col], errors="coerce").interpolate(
            method="time", limit_direction="both"
        )

        # Direction circular interpolation
        theta = np.deg2rad(pd.to_numeric(df[dir_col], errors="coerce"))
        sin_i = pd.Series(np.sin(theta), index=df.index).interpolate(method="time", limit_direction="both")
        cos_i = pd.Series(np.cos(theta), index=df.index).interpolate(method="time", limit_direction="both")
        r = np.hypot(sin_i, cos_i)
        with np.errstate(invalid="ignore", divide="ignore"):
            df[dir_col] = (np.degrees(np.arctan2(sin_i / r, cos_i / r)) % 360.0)

        after = df[[dir_col, spd_col]].isna().sum()
        logger.info("NaNs after  | %s=%d, %s=%d",
                    dir_col, int(after[dir_col]),
                    spd_col, int(after[spd_col]))
        resource_snapshot("after_fill")
    return df



In [ ]:
from __future__ import annotations

import logging
import numpy as np
import pandas as pd

logger = logging.getLogger(__name__)

# --------------------------------------------------------------------
# Helpers
# --------------------------------------------------------------------
def prep_sr_timestamps(
    df_sr: pd.DataFrame,
    *,
    ts_col: str = "creation_timestamp",
    out_round_col: str = "creation_rounded",
    cutoff: pd.Timestamp = pd.Timestamp("2020-12-31 23:00"),
) -> pd.DataFrame:
    """
    Prepare SR timestamps:
      - parse ts, drop invalid
      - make tz-naive
      - out_round_col rule:
          if ts < cutoff  -> round to 30min
          if ts >= cutoff -> set exactly to cutoff
      - return filtered copy with valid out_round_col
    """
    sr = df_sr.copy()
    sr[ts_col] = pd.to_datetime(sr[ts_col], errors="coerce")
    n_bad = sr[ts_col].isna().sum()
    if n_bad:
        logger.info("Dropped %d SR rows with invalid %s", n_bad, ts_col)

    # Make tz-naive BEFORE comparisons/rounding
    try:
        sr[ts_col] = sr[ts_col].dt.tz_localize(None)
    except (TypeError, AttributeError):
        pass

    # Apply rounding/cutoff rule
    sr[out_round_col] = sr[ts_col].where(
        sr[ts_col] >= cutoff,
        sr[ts_col].dt.round("30min")
    ).mask(sr[ts_col] >= cutoff, cutoff)

    # Make rounded col tz-naive (defensive)
    try:
        sr[out_round_col] = sr[out_round_col].dt.tz_localize(None)
    except (TypeError, AttributeError):
        pass

    # Keep only rows with a valid rounded timestamp
    sr = sr.loc[sr[out_round_col].notna()].copy()

    n_forced = (sr[ts_col] >= cutoff).sum()
    if n_forced:
        logger.info("Forced %d SR timestamps to cutoff %s", n_forced, cutoff)

    logger.info("SR rows retained: %d", len(sr))
    return sr


def build_half_hour_grid_with_circular_dir(aqm: pd.DataFrame) -> pd.DataFrame:
    """
    Build a 30‑minute grid from hourly AQM readings:
      - :00 (original) kept as-is
      - :30 is arithmetic mean of t and t+1 for all numeric columns,
        EXCEPT wind direction uses circular interpolation.
      - Wind speed remains a simple arithmetic average (old behavior).
    Column names assumed:
      - "wind direction degree"   (degrees 0..360)
      - "wind speed m/s"          (m/s)
    """
    aqm_00 = aqm

    # Start with arithmetic half-hour means for all numeric columns
    aqm_30 = (aqm + aqm.shift(-1)) / 2

    # Circular interpolation for wind direction ONLY (no speed-weighting)
    if "wind direction degree" in aqm.columns:
        d0 = np.deg2rad(aqm["wind direction degree"])
        d1 = np.deg2rad(aqm["wind direction degree"].shift(-1))

        valid = d0.notna() & d1.notna()

        u0, v0 = np.cos(d0), np.sin(d0)
        u1, v1 = np.cos(d1), np.sin(d1)
        u_half = (u0 + u1) / 2.0
        v_half = (v0 + v1) / 2.0

        dir_half = (np.degrees(np.arctan2(v_half, u_half)) + 360.0) % 360.0
        dir_half = dir_half.where(valid)  # keep NaN if either endpoint is NaN

        # Overwrite arithmetic mean for direction with circular result
        aqm_30["wind direction degree"] = dir_half

    # Index for half-hour
    aqm_30.index = aqm_30.index + pd.Timedelta(minutes=30)

    # Combine :00 and :30
    aqm_30min = pd.concat([aqm_00, aqm_30]).sort_index()
    logger.info("AQM 30‑min grid rows: %d", len(aqm_30min))
    return aqm_30min


# --------------------------------------------------------------------
# Orchestrator
# --------------------------------------------------------------------
def join_sr_to_aqm_fast(
    df_sr: pd.DataFrame,
    df_aqm: pd.DataFrame,
    ts_col: str = "creation_timestamp",
    out_round_col: str = "creation_rounded",
    cutoff: pd.Timestamp = pd.Timestamp("2020-12-31 23:00"),
) -> pd.DataFrame:
    """
    Vectorized join of SR to AQM:
      - SR timestamps parsed & rounded via `prep_sr_timestamps`
      - AQM hourly → 30‑min grid via `build_half_hour_grid_with_circular_dir`
      - Coerce AQM to numeric (non-numeric → NaN)
      - Merge on 30‑min timestamps
    """
    with timed("join_sr_to_aqm_fast:TOTAL"):
        # ---------- SR prep ----------
        with timed("join_sr_to_aqm_fast:SR_preparation"):
            sr = prep_sr_timestamps(
                df_sr, ts_col=ts_col, out_round_col=out_round_col, cutoff=cutoff
            )
            resource_snapshot("join_sr_to_aqm_fast:after_SR_prep")

        # ---------- AQM prep ----------
        with timed("join_sr_to_aqm_fast:AQM_preparation"):
            aqm = df_aqm.copy()
            aqm.index = pd.to_datetime(aqm.index, errors="coerce")
            aqm = aqm.loc[aqm.index.notna()].sort_index()

            try:
                aqm.index = aqm.index.tz_localize(None)
            except (TypeError, AttributeError):
                pass

            if aqm.index.has_duplicates:
                dup_cnt = aqm.index.duplicated().sum()
                logger.info("Found %d duplicate AQM timestamps; aggregating with mean", dup_cnt)
                aqm = aqm.groupby(aqm.index).mean(numeric_only=True)

            logger.info("AQM rows after datetime prep: %d", len(aqm))
            resource_snapshot("join_sr_to_aqm_fast:after_AQM_prep")

        # ---------- Numeric coercion ----------
        with timed("join_sr_to_aqm_fast:AQM_numeric_coercion"):
            non_numeric_before = (~aqm.stack().map(np.isreal)).sum()
            aqm = aqm.apply(pd.to_numeric, errors="coerce")
            total_nans_after = aqm.isna().sum().sum()
            logger.info(
                "AQM non‑numeric→NaN approx: %d; total NaNs after coercion: %d",
                non_numeric_before, total_nans_after
            )
            if total_nans_after > 0:
                nan_rows = aqm[aqm.isna().any(axis=1)].head(5)
                logger.info("Sample NaN rows (up to 5):\n%s", nan_rows)
            resource_snapshot("join_sr_to_aqm_fast:after_numeric")

        # ---------- Build 30‑minute grid (circular dir, speed arithmetic) ----------
        with timed("join_sr_to_aqm_fast:build_30min_grid"):
            aqm_30min = build_half_hour_grid_with_circular_dir(aqm)
            resource_snapshot("join_sr_to_aqm_fast:after_grid")

        # ---------- Join ----------
        with timed("join_sr_to_aqm_fast:merge"):
            out = sr.merge(aqm_30min, left_on=out_round_col, right_index=True, how="left")
            logger.info("Join output shape: %s", out.shape)

            # quick visibility on unmatched rows (all-NaN AQM fields)
            unmatched = out[out.isna().all(axis=1)].shape[0]
            if unmatched:
                logger.info("Rows with all NaNs after join (possible no AQM match): %d", unmatched)

            resource_snapshot("join_sr_to_aqm_fast:after_join")

    return out


# This augments filtered Bellville South service requests with wind data

In [ ]:
df_bv_wind_processed = process_wind_data("../data/Wind_direction_and_speed_2020.ods", site_name="Bellville")
df_std = standardize_wind_column_names(df_bv_wind_processed, site_name="Bellville")
save_df_to_csv(df_std, "bv_wind_processed.csv", True)

In [ ]:

df_filled = fill_wind_missing(df_std)
save_df_to_csv(df_filled, "bv_wind_filled.csv", True)

In [ ]:
sr_with_wind = join_sr_to_aqm_fast(df_sr=df_1min_bellville, df_aqm=df_filled)
sr_with_wind["completion_timestamp"] = pd.to_datetime(sr_with_wind["completion_timestamp"], errors="coerce")
sr_with_wind["completion_timestamp"] = sr_with_wind["completion_timestamp"].dt.tz_localize(None)
save_df_to_csv(sr_with_wind, "sr_with_wind.csv")

# Anonymize augmented subsample


In [ ]:
import numpy as np
import pandas as pd

def anonymize_sr_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Anonymizes a service‑request DataFrame by:
      - Adding Entry_number = 1..N (surrogate ID).
      - Dropping direct identifiers and precise coordinates/time‑derived fields.
      - Generalizing timestamps to 6‑hour blocks (tz‑naive).
      - Keeping spatial context via H3 index only (no raw lat/lon).

    Preserves existing timers/logging/resource snapshots.
    """
    columns_to_drop = [
        "notification_number",
        "reference_number",
        "distance_km",
        "creation_rounded",
        "latitude",
        "longitude",
        "code",
        "cause_code",
    ]

    with timed("anonymize_sr_data:TOTAL"):
        with timed("anonymize_sr_data:drop_columns"):
            df_anonymized = df.copy()

            # Add sequential surrogate ID (1..N) before dropping anything
            df_anonymized.insert(0, "Entry_number", np.arange(1, len(df_anonymized) + 1, dtype=np.int64))

            present = [c for c in columns_to_drop if c in df_anonymized.columns]
            missing = [c for c in columns_to_drop if c not in df_anonymized.columns]
            if missing:
                logger.debug("Columns not present (ignored): %s", missing)

            df_anonymized = df_anonymized.drop(columns=present, errors="ignore")
            logger.info("Dropped %d columns; resulting cols=%d", len(present), df_anonymized.shape[1])
            resource_snapshot("anonymize_sr_data:after_drop")

        with timed("anonymize_sr_data:timestamp_generalization"):
            # Generalize to 6‑hour blocks, keep tz‑naive for consistent joins/storage
            for c in ("creation_timestamp", "completion_timestamp"):
                if c in df_anonymized.columns:
                    df_anonymized[c] = pd.to_datetime(df_anonymized[c], errors="coerce")
                    try:
                        df_anonymized[c] = df_anonymized[c].dt.tz_localize(None)
                    except (TypeError, AttributeError):
                        # Already tz‑naive or column missing dtype support
                        pass
                    df_anonymized[c] = df_anonymized[c].dt.floor("6h")

            # quick stat on rows affected
            n_na_create = df_anonymized["creation_timestamp"].isna().sum() if "creation_timestamp" in df_anonymized else 0
            n_na_complete = df_anonymized["completion_timestamp"].isna().sum() if "completion_timestamp" in df_anonymized else 0
            logger.info("Timestamp NaNs | creation=%d, completion=%d", n_na_create, n_na_complete)
            resource_snapshot("anonymize_sr_data:after_time_gen")

        logger.info("Anonymized SR shape: %s", df_anonymized.shape)

    return df_anonymized


In [ ]:
sr_anonymized = anonymize_sr_data(sr_with_wind)
save_df_to_csv(sr_anonymized, "anon_sr_data.csv")

The function above anonymizes a service request DataFrame by removing identifying columns and reducing temporal precision. Spatial data is generalized by keeping only H3 index. H3 index was chosen at resolution 8 since it features an average length of 461 meters and farthest distance from the centroid is 533 meters, this level of granularity is fine for the purpose of this exercise then.

Columns dropped were notification number, reference number, distance km, creation rounded, latitude, longitude, code, and cause code. Distance km is something that I created for calcualting the 1 minute from the centroid of Bellville. Code and cause were dropped as they seem to be pretty detailed and can make individual complaints unique and reduces identification risk.

# Unit tests

In [ ]:
%%ipytest -q

# ---- Utilities ---------------------------------------------------------------

def _make_tmp_tree(tmp_path: Path, name="ds_code_challenge"):
    """Create a fake project root with a /data dir and return (root, data_dir)."""
    root = tmp_path / name
    data = root / "data"
    data.mkdir(parents=True, exist_ok=True)
    return root, data

# ---- Basic helpers: ensure_dir / parent_dir_of / fmt_bytes -------------------

def test_ensure_dir_creates_directory(tmp_path):
    target = tmp_path / "a" / "b" / "c"
    assert not target.exists()
    nb.ensure_dir(target)
    assert target.exists() and target.is_dir()

@pytest.mark.parametrize(
    "inp,expect",
    [
        ("/a/b/c.txt", "/a/b"),
        ("/a/b/", "/a/b"),
        ("justafile", "."),
        ("", "."),
    ],
)
def test_parent_dir_of(inp, expect):
    got = nb.parent_dir_of(inp)
    assert got == expect

@pytest.mark.parametrize(
    "n,expect",
    [
        (0, "0.0B"),
        (10, "10.0B"),
        (1023, "1023.0B"),
        (1024, "1.0KB"),
        (1024**2 * 1.5, "1.5MB"),
        ("bad", "bad"),  # passthrough on exception
    ],
)
def test_fmt_bytes(n, expect):
    assert nb.fmt_bytes(n) == expect

# ---- gunzip_file -------------------------------------------------------------

def test_gunzip_file_roundtrip(tmp_path):
    # Create gz source
    src = tmp_path / "x.csv.gz"
    dst = tmp_path / "out.csv"
    raw = b"col1,col2\n1,2\n3,4\n"
    with gzip.open(src, "wb") as f:
        f.write(raw)

    nb.gunzip_file(str(src), str(dst))
    assert dst.exists()
    assert dst.read_bytes() == raw

# ---- client_error_msg --------------------------------------------------------

class _FakeBotoError(Exception):
    def __init__(self, code=None, message=None):
        self.response = {"Error": {}}
        if code is not None:
            self.response["Error"]["Code"] = code
        if message is not None:
            self.response["Error"]["Message"] = message
        super().__init__(message or "")

@pytest.mark.parametrize(
    "code,msg,expect_regex",
    [
        ("AccessDenied", "Nope", r"\[AccessDenied\] Nope"),
        (None, "OnlyMsg", r"OnlyMsg"),
        (None, None, r"^$"),  # falls back to str(e) which is empty here
    ],
)
def test_client_error_msg_formats(code, msg, expect_regex):
    e = _FakeBotoError(code, msg)
    out = nb.client_error_msg(e)
    assert re.search(expect_regex, out) is not None

# ---- timed() & resource_snapshot() ------------------------------------------

def test_timed_logs_elapsed(caplog):
    caplog.set_level("INFO")
    with nb.timed("unit-test"):
        time.sleep(0.01)
    # Look for "... took X.XXX s"
    assert any("unit-test took " in rec.getMessage() for rec in caplog.records)

def test_resource_snapshot_safe_when_not_debug(caplog):
    caplog.set_level("INFO")
    nb.resource_snapshot("noop")  # should not raise, may log nothing
    assert True  # reached

# ---- resolve_path ------------------------------------------------------------

def test_resolve_path_with_rel_and_default(tmp_path, monkeypatch):
    # Default DEST_ROOT path
    monkeypatch.setattr(nb, "DEST_ROOT", str(tmp_path / "data"))
    p1 = nb.resolve_path(None, "file.csv")
    assert p1 == tmp_path / "data" / "file.csv"

    p2 = nb.resolve_path(str(tmp_path / "alt"), "file.csv")
    assert p2 == tmp_path / "alt" / "file.csv"

# ---- save_df_to_csv (first definition with rel arg) --------------------------
# If your notebook overwrote save_df_to_csv later, this test is skipped.
save_has_rel = "rel" in nb.save_df_to_csv.__code__.co_varnames

@pytest.mark.skipif(not save_has_rel, reason="save_df_to_csv(rel=...) not present (overwritten later).")
def test_save_df_to_csv_with_rel(tmp_path, caplog, monkeypatch):
    caplog.set_level("INFO")
    monkeypatch.setattr(nb, "DEST_ROOT", str(tmp_path / "data"))
    df = pd.DataFrame({"a": [1, 2]})
    nb.save_df_to_csv(df, "test.csv", rel=str(tmp_path / "out"))
    out = tmp_path / "out" / "test.csv"
    assert out.exists()
    got = pd.read_csv(out)
    pd.testing.assert_frame_equal(got, df)

# ---- save_df_to_csv (second definition without rel arg) ----------------------

@pytest.mark.skipif(save_has_rel, reason="Testing the non-rel version only when it overwrote the first.")
def test_save_df_to_csv_no_rel(tmp_path, monkeypatch):
    # Fake cwd so function saves under <cwd>/.. / output
    class _FakePath(Path):
        _flavour = Path(".")._flavour
    def _fake_cwd():
        return _FakePath(str(tmp_path / "proj" / "work"))
    (tmp_path / "proj" / "work").mkdir(parents=True, exist_ok=True)

    monkeypatch.setattr(Path, "cwd", staticmethod(_fake_cwd))
    df = pd.DataFrame({"x": [1, 2, 3]})
    nb.save_df_to_csv(df, "alpha.csv")
    expected = tmp_path / "proj" / "output" / "alpha.csv"
    assert expected.exists()
    pd.testing.assert_frame_equal(pd.read_csv(expected), df)

# ---- download_file (mock network) -------------------------------------------

def test_download_file_streams_bytes(tmp_path, monkeypatch, caplog):
    caplog.set_level("INFO")
    monkeypatch.setattr(nb, "DEST_ROOT", str(tmp_path / "data"))

    # Fake requests.get
    class _Resp:
        def __init__(self, content: bytes):
            self._bio = io.BytesIO(content)
            self.status_code = 200
        def raise_for_status(self): pass
        def iter_content(self, chunk_size=8192):
            while True:
                chunk = self._bio.read(chunk_size)
                if not chunk: break
                yield chunk
        def __enter__(self): return self
        def __exit__(self, *a): pass

    def _fake_get(url, stream=True, timeout=60):
        return _Resp(b"hello world")

    monkeypatch.setattr(nb.requests, "get", _fake_get)
    nb.download_file("http://example.com/x.bin", filename="x.bin")
    out = Path(nb.DEST_ROOT) / "x.bin"
    assert out.exists() and out.read_bytes() == b"hello world"

# ---- fetch_centroid_latlon (mock ArcGIS + geopandas) -------------------------

@pytest.mark.parametrize("multi_feature", [False, True])
def test_fetch_centroid_latlon_mock(monkeypatch, multi_feature):
    try:
        import geopandas as gpd
        from shapely.geometry import Polygon
    except Exception:
        pytest.skip("geopandas/shapely not available")

    features = [{
        "type":"Feature",
        "geometry":{"type":"Polygon","coordinates":[[(0,0),(1,0),(1,1),(0,1),(0,0)]]},
        "properties":{}
    }]
    if multi_feature:
        features = features * 2
    fake_json = {"features": features}

    class _Resp:
        def raise_for_status(self): pass
        def json(self): return fake_json

    monkeypatch.setattr(nb.requests, "get", lambda *a, **k: _Resp())

    # Use a projected CRS to avoid centroid-on-geographic warnings
    lat, lon = nb.fetch_centroid_latlon(
        suburb_name="ANY",
        featureserver_url="http://fake",
        epsg_utm=3857,
        req_timeout=1,
    )
    assert pytest.approx(lat, abs=1e-3) == 0.5
    assert pytest.approx(lon, abs=1e-3) == 0.5

# ---- haversine_km ------------------------------------------------------------

def test_haversine_km_basic():
    # Distance from same point is ~0
    d0 = nb.haversine_km(0, 0, 0, 0)
    assert d0 == pytest.approx(0.0, abs=1e-9)

    # Rough Cape Town City Hall to Table Mountain Lower Cableway
    ct_hall = (-33.9253, 18.4222)
    cable = (-33.9703, 18.4031)
    d = nb.haversine_km(ct_hall[0], ct_hall[1], cable[0], cable[1])
    assert 4.0 < d < 7.5  # sanity band (~5.5 km)

# ---- filter_within_radius_km -------------------------------------------------

def test_filter_within_radius_km_filters_and_option(tmp_path, caplog):
    caplog.set_level("INFO")
    cent = (-33.9249, 18.4241)  # Cape Town CBD approx
    df = pd.DataFrame({
        "latitude":  [cent[0], cent[0] + 0.02, np.nan],
        "longitude": [cent[1], cent[1] + 0.02, 18.5],
        "id": [1, 2, 3],
    })
    out = nb.filter_within_radius_km(df, cent[0], cent[1], radius_km=1.852, keep_distance_col=False)
    # ~0.02 deg lat ~ 2.22 km => should exclude id=2, keep id=1 only
    assert list(out["id"]) == [1]
    assert "distance_km" not in out.columns

# ---- _infer_site_wind_cols ---------------------------------------------------

def test_infer_site_wind_cols_picks_shortest_match():
    cols = [
        "DateTime",
        "Bellville_South_AQM_Site_Deg",
        "Bellville_South_AQM_Site_m/s",
        "Bellville_South_AQM_Site_Deg_extra",
    ]
    df = pd.DataFrame(columns=cols)
    dcol, scol = nb._infer_site_wind_cols(df, "Bellville_South")
    assert dcol == "Bellville_South_AQM_Site_Deg"
    assert scol == "Bellville_South_AQM_Site_m/s"

# ---- standardize_wind_column_names / fill_wind_missing -----------------------

def test_standardize_and_fill_wind_missing():
    rng = pd.date_range("2020-01-01", periods=5, freq="h")
    df = pd.DataFrame({
        "DateTime": rng,
        "Bellville_South_AQM_Site_Deg": [0, np.nan, 90, np.nan, 180],
        "Bellville_South_AQM_Site_m/s": [1.0, np.nan, 2.0, np.nan, 3.0],
    }).set_index("DateTime")

    std = nb.standardize_wind_column_names(df, "Bellville_South")
    assert {"wind direction degree", "wind speed m/s"} <= set(std.columns)

    filled = nb.fill_wind_missing(std.copy())
    # After time interpolation, the NaNs should be filled
    assert filled["wind speed m/s"].isna().sum() == 0
    assert filled["wind direction degree"].isna().sum() == 0

# =============================================================================
# NEW/UPDATED TESTS FOR REFACTORED HELPERS
# =============================================================================

def test_prep_sr_timestamps_rounding_and_cutoff():
    sr = pd.DataFrame({
        "creation_timestamp": ["2020-12-31 22:40", "2021-01-01 08:00", "bad-ts"]
    })
    out = nb.prep_sr_timestamps(sr, ts_col="creation_timestamp",
                                out_round_col="creation_rounded",
                                cutoff=pd.Timestamp("2020-12-31 23:00"))

    # invalid ts dropped
    assert len(out) == 2
    # first < cutoff -> round to 22:30
    assert pd.Timestamp(out.loc[out.index[0], "creation_rounded"]) == pd.Timestamp("2020-12-31 22:30")
    # second >= cutoff -> forced to 23:00
    assert pd.Timestamp(out.loc[out.index[1], "creation_rounded"]) == pd.Timestamp("2020-12-31 23:00")

def test_build_half_hour_grid_with_circular_dir_behaviour():
    # Two consecutive hourly records:
    # 22:00: dir=350°, speed=3
    # 23:00: dir=10°,  speed=1
    # At 22:30 we expect circular average dir ≈ 0°, and speed = arithmetic mean = 2.0
    idx = pd.to_datetime(["2020-12-31 22:00", "2020-12-31 23:00"])
    aqm = pd.DataFrame({
        "wind direction degree": [350.0, 10.0],
        "wind speed m/s": [3.0, 1.0],
        "other": [100.0, 200.0],
    }, index=idx)

    grid = nb.build_half_hour_grid_with_circular_dir(aqm)

    # :00 rows preserved
    assert pd.Timestamp("2020-12-31 22:00") in grid.index
    assert pd.Timestamp("2020-12-31 23:00") in grid.index
    # :30 row created
    mid = grid.loc[pd.Timestamp("2020-12-31 22:30")]
    # circular direction ≈ 0 (or 360)
    assert (abs(((mid["wind direction degree"] - 0) + 180) % 360 - 180)) < 1e-6
    # speed arithmetic mean
    assert mid["wind speed m/s"] == pytest.approx(2.0, 1e-9)
    # other numeric columns arithmetic mean
    assert mid["other"] == pytest.approx((100 + 200) / 2.0, 1e-9)

def test_build_half_hour_grid_with_circular_dir_nan_semantics():
    # If one endpoint dir is NaN, :30 dir should be NaN (preserve old mean semantics)
    idx = pd.to_datetime(["2020-01-01 00:00", "2020-01-01 01:00"])
    aqm = pd.DataFrame({
        "wind direction degree": [np.nan, 90.0],
        "wind speed m/s": [2.0, 4.0],
    }, index=idx)

    grid = nb.build_half_hour_grid_with_circular_dir(aqm)
    mid = grid.loc[pd.Timestamp("2020-01-01 00:30")]

    assert np.isnan(mid["wind direction degree"])
    # speed arithmetic mean -> NaN if one is NaN? (here both speeds present -> 3.0)
    assert mid["wind speed m/s"] == pytest.approx(3.0, 1e-9)

# ---- join_sr_to_aqm_fast -----------------------------------------------------

def test_join_sr_to_aqm_fast_minimal_updated(caplog):
    caplog.set_level("INFO")
    # SR: two times, one before cutoff (round to :30), one after (forced to cutoff)
    sr = pd.DataFrame({"creation_timestamp": ["2020-12-31 22:40", "2021-01-01 08:00"]})

    # AQM hourly: provide speed/dir on the hour
    idx = pd.date_range("2020-12-31 21:00", periods=4, freq="h")
    aqm = pd.DataFrame({
        "wind direction degree": [300.0, 350.0, 10.0, 60.0],
        "wind speed m/s": [1.0, 2.0, 3.0, 4.0],
    }, index=idx)

    out = nb.join_sr_to_aqm_fast(sr, aqm, cutoff=pd.Timestamp("2020-12-31 23:00"))

    # Expect two rows out
    assert len(out) == 2
    assert "creation_rounded" in out.columns

    # First SR (22:40) -> round("30min") -> 22:30; speed should be arithmetic mean (2.0 and 3.0 -> 2.5)
    first = out.iloc[0]
    assert pd.to_datetime(first["creation_rounded"]) == pd.Timestamp("2020-12-31 22:30")
    assert first["wind speed m/s"] == pytest.approx((2.0 + 3.0) / 2.0, 1e-9)

    # Direction at 22:30 should circular‑average 350° and 10° -> ≈ 0°
    assert (abs(((first["wind direction degree"] - 0) + 180) % 360 - 180)) < 1e-6

    # Second SR (>= cutoff) -> forced to cutoff 23:00; should match 23:00 row exactly
    second = out.iloc[1]
    assert pd.to_datetime(second["creation_rounded"]) == pd.Timestamp("2020-12-31 23:00")
    assert second["wind speed m/s"] == pytest.approx(3.0, 1e-9)
    assert second["wind direction degree"] == pytest.approx(10.0, 1e-9)

# ---- anonymize_sr_data -------------------------------------------------------

def test_anonymize_sr_data_drops_cols_and_blocks_time():
    df = pd.DataFrame({
        "notification_number":[1],
        "reference_number":[2],
        "distance_km":[0.5],
        "creation_rounded":["2020-01-01 00:00"],
        "latitude":[-33.9],
        "longitude":[18.4],
        "code":["X"],
        "cause_code":["Y"],
        "h3_level8_index":["88ad..."],
        "creation_timestamp":["2020-01-01 01:23:45+02:00"],
        "completion_timestamp":["2020-01-01 06:01:00+02:00"],
        "other":[123],
    })
    out = nb.anonymize_sr_data(df)

    # Dropped columns
    for c in ["notification_number","reference_number","distance_km",
              "creation_rounded","latitude","longitude","code","cause_code"]:
        assert c not in out.columns

    # Timestamps floored to 6h blocks (timezone-naive after processing)
    c0 = pd.to_datetime(out.loc[0, "creation_timestamp"])
    c1 = pd.to_datetime(out.loc[0, "completion_timestamp"])
    assert c0 == pd.Timestamp("2020-01-01 00:00")
    assert c1 == pd.Timestamp("2020-01-01 06:00")

    # Preserved other fields
    assert out.loc[0, "h3_level8_index"] == "88ad..."
    assert out.loc[0, "other"] == 123
